In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load data
q2_data = np.load('q2_data/q2_data.npz')

In [2]:
# Extract training and test sets:
X_train = q2_data['q2x_train']
y_train = q2_data['q2y_train']
X_test = q2_data['q2x_test']
y_test = q2_data['q2y_test']

Check the classes of y:

In [3]:
unique_labels = list(set(y_train.flatten().tolist()))
print(unique_labels)
K = len(unique_labels)
print("There are {} classes for the labels.".format(K))

[1.0, 2.0, 3.0]
There are 3 classes for the labels.


Now implement the softmax regression from pieces:

In [15]:
# exponentional:
def exponential(X, w_k):
    return np.exp(X.dot(w_k))


# probability:
def probability(X, w, k):
    exp_list = [exponential(X, w[:,j]) for j in range(K-1)]
    denominator = 1 + np.sum(exp_list)
    if k == K - 1:
        return 1 / denominator
    nominator = exponential(X, w[:,k])
    return nominator / denominator
    
    
# indicator function:
def indicator(y_true, m):
    if y_true == m: 
        return 1
    return 0


# log-likelihood function:
def log_likelihood(X, w, y):
    ll = 0
    for i in range(X.shape[0]):
        for k in range(K):
            if y[i] == unique_labels[k]:
                ll += np.log(probability(X[i,:], w, k))
    return ll
            
    
# gradient:
def gradient(X, w, y, k):
    l = X.shape[0]
    grad = 0
    for i in range(l):
        grad += X[i,:] * (indicator(y[i], unique_labels[k]) - probability(X[i,:], w, k))
    return grad
        

# training：
def training(X, w, y):
    alpha = 0.0005
    epsilon = 1e-10
    ll = 0
    ll_new = 1
    num_iter = 100
    curr_iter = 0
    while abs(ll - ll_new) > epsilon and curr_iter < num_iter:
        print("w:{}".format(w))
        curr_iter += 1
        ll = ll_new
        grad_matrix = np.zeros((w.shape[0], w.shape[1]))
        for k in range(K-1):
            grad_matrix[:,k] = gradient(X, w, y, k)
        print(grad_matrix, grad_matrix.shape)
        w += alpha * grad_matrix
        ll_new = log_likelihood(X, w, y)
        print("The log likelihood at the iteration {} is: {}".format(curr_iter, ll_new))
    print("The final result for the weights are: ", w)
    
        
# print(X_train.shape)
# w = np.zeros((X_train.shape[1], K - 1))
# print(w.shape)
# print(log_likelihood(X_train, w, y_train))
# training(X_train, w, y_train)

In [5]:
def predict(X, w, y):
    count = 0
    for i in range(len(y)):
        probability_list = []
        for k in range(K):
            probability_list.append(probability(X[i,:], w, k))
        largest_index = probability_list.index(max(probability_list))
        predicted_y = unique_labels[largest_index]
#         print("Predict the {}-th x_test as {}".format(i, predicted_y))
        if predicted_y == y[i]:
            count += 1
    accuracy = count / len(y)
    print("The accuracy of the test data is {}".format(accuracy))
    
predict(X_test, w, y_test)

The accuracy of the test data is 0.92


Now compare with the baseline: logistic regression from sklearn

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(C = 1e5)
clf = logreg.fit(X_train, y_train)
predicted_y = clf.predict(X_test)
accuracy_score(predicted_y, y_test)

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.96

In [16]:
x=np.array([[0,7,0,4],[4,5,6,7]])
x = np.reshape(x,(4,2))
y=np.array([[1],[1],[2],[3]]) 
w = np.zeros((x.shape[1], 2))

In [17]:
training(x,w,y)

w:[[0. 0.]
 [0. 0.]]
[[-3.33333333  0.66666667]
 [ 3.33333333 -2.66666667]] (2, 2)
The log likelihood at the iteration 1 is: -4.37962132319119
w:[[-0.00166667  0.00033333]
 [ 0.00166667 -0.00133333]]
[[-3.34429324  0.6830173 ]
 [ 3.28646067 -2.61587813]] (2, 2)
The log likelihood at the iteration 2 is: -4.365032630189109
w:[[-0.00333881  0.00067484]
 [ 0.0033099  -0.00264127]]
[[-3.35463798  0.69869367]
 [ 3.24072655 -2.56650855]] (2, 2)
The log likelihood at the iteration 3 is: -4.350673252985281
w:[[-0.00501613  0.00102419]
 [ 0.00493026 -0.00392453]]
[[-3.36438496  0.71371896]
 [ 3.19610483 -2.51851403]] (2, 2)
The log likelihood at the iteration 4 is: -4.336533897670807
w:[[-0.00669832  0.00138105]
 [ 0.00652831 -0.00518378]]
[[-3.37355122  0.72811558]
 [ 3.15256962 -2.47185203]] (2, 2)
The log likelihood at the iteration 5 is: -4.322605769871098
w:[[-0.0083851   0.00174511]
 [ 0.0081046  -0.00641971]]
[[-3.3821535   0.74190513]
 [ 3.11009533 -2.42648131]] (2, 2)
The log likelihood

The log likelihood at the iteration 92 is: -3.502789030139251
w:[[-0.15440203  0.04201126]
 [ 0.09945263 -0.06723451]]
[[-3.13268466  0.90635352]
 [ 1.53877007 -0.85638389]] (2, 2)
The log likelihood at the iteration 93 is: -3.4959307776312505
w:[[-0.15596837  0.04246444]
 [ 0.10022201 -0.0676627 ]]
[[-3.12574815  0.90374498]
 [ 1.5319265  -0.85037023]] (2, 2)
The log likelihood at the iteration 94 is: -3.4891121498761613
w:[[-0.15753125  0.04291631]
 [ 0.10098798 -0.06808789]]
[[-3.11880244  0.90111303]
 [ 1.52517938 -0.84446099]] (2, 2)
The log likelihood at the iteration 95 is: -3.482332824636738
w:[[-0.15909065  0.04336687]
 [ 0.10175057 -0.06851012]]
[[-3.11184869  0.8984588 ]
 [ 1.51852627 -0.83865363]] (2, 2)
The log likelihood at the iteration 96 is: -3.4755924842818553
w:[[-0.16064657  0.0438161 ]
 [ 0.10250983 -0.06892944]]
[[-3.10488801  0.89578336]
 [ 1.51196481 -0.83294567]] (2, 2)
The log likelihood at the iteration 97 is: -3.4688908156331673
w:[[-0.16219901  0.04426399]
